# Threshold Joye-Libert Secure Aggregation

In [8]:
from ftsa.protocols.buildingblocks.JoyeLibert import TJLS
import random

t = 412
threshold = 7
nclients = 10
drops = 3
keysize = 2048 
inputsize = 16
dimension = 100

users = list(range(1, nclients+1))
alive = list(range(1, nclients-drops+1))
dropped = list(range(nclients-drops+1,nclients+1))


TJL = TJLS(nclients, threshold)

print("users: ", users)
print("alive: ", alive)
print("dropped: ", dropped)

users:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
alive:  [1, 2, 3, 4, 5, 6, 7]
dropped:  [8, 9, 10]


### Generate JL keys (**Setup**)

In [9]:
# generate keys
pp, skey, ukey = TJL.Setup(keysize)

print(" - Public Parameters:", pp)
print(" - Aggregator Key:", skey)
print(" - User Keys:", ukey)

 - Public Parameters: <PublicParam (N=98069...35673, H(x)=0x3bf0)>
 - Aggregator Key: <ServerKey -0xd00e24b>
 - User Keys: {0: <UserKey 0x9e4be92b>, 1: <UserKey 0x184b9226>, 2: <UserKey 0x1f69e563>, 3: <UserKey 0x20e755fe>, 4: <UserKey 0xea24189f>, 5: <UserKey 0x1df5a992>, 6: <UserKey 0x1a4b6c45>, 7: <UserKey 0x1bcc3ee0>, 8: <UserKey 0x1648eeb8>, 9: <UserKey 0x124d3fde>}


### Share the JL user key with all other users (**SKShare**)


In [10]:

# share secret key of each user
shares = {}
for u in users: 
    shares[u] = TJL.SKShare(ukey[u-1],threshold, users)

# distribute the shares
from collections import defaultdict
usersshares = defaultdict(dict)
for v in users:
    for u in users: 
        usersshares[u][v] = shares[v][u-1]

### Protect inputs of alive users with the users' keys (**Protect**)

In [11]:
# generate $nclients random numbers
l =[]
for i in range(0,nclients):
    l.append(random.randint(0,2**16))

print("Clients Inputs: ", l )


# protect each number with a different client key
e=[]
for u in alive:
    e.append(TJL.Protect(pp, ukey[u-1], t, l[u-1]))

e

Clients Inputs:  [1975, 22868, 19429, 28513, 59938, 43893, 34926, 24075, 7319, 65109]


[<EncryptedNumber 20905...20629>,
 <EncryptedNumber 65188...64970>,
 <EncryptedNumber 10549...95268>,
 <EncryptedNumber 49406...56145>,
 <EncryptedNumber 17360...00069>,
 <EncryptedNumber 65186...20235>,
 <EncryptedNumber 29167...52651>]

### Protect the zero-value with the key shares of the dropped users  (**ShareProtect**)

In [12]:
if dropped != []:
    yzeroshare ={}
    for u in alive:
        dropped_users_shares = []
        for v in dropped:
            dropped_users_shares.append(usersshares[u][v])
        yzeroshare[u] = TJL.ShareProtect(pp, dropped_users_shares,t)


### Combine shares of the protected zero-value (**ShareCombine**)

In [13]:
if dropped != []:
    yzero = TJL.ShareCombine(pp, list(yzeroshare.values()), threshold)
    print("yzero:", yzero)
else:
    yzero = None

yzero: <EncryptedNumber 41268...64142>


### Aggregate all user protected inputs and the protected zero-value of dropped users (**Agg**)

In [14]:
s = TJL.Agg(pp, skey, t, e, yzero)


print("Sum=", s )
# check if the result is correct
print("Verify=", s == sum(l[:nclients-drops]))

Sum= 211542
Verify= True
